In [ ]:
# done done done

In [1]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *
    
# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'  # Europe
site = 'eng_yidaiyilu_gov_cn'
c = Crawl()  # creating object

# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

# create image directories
image_directory = c.create_image_directories(project_path)

# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_article_url = 0
unable_to_fetch_rss_url = 0
publish_source = 'eng.yidaiyilu.gov.cn'
country = 'China'
language = 'English'
images_path = []  
cookies = {
    '__jsluid_h': 'b9a39f29122a5b502b2feefa25d5199b',
    '__jsluid_s': 'aaa9d176c16846ce7dec2e29fc57151e',
    'yfx_c_g_u_id_10008969': '_ck21032016220114299131327555651',
    'yfx_f_l_v_t_10008969': 'f_t_1616237521304__r_t_1617787482968__v_t_1617787482968__r_c_3',
    'Hm_lvt_25e78b3fa4b036e241d0874f836fb377': '1616237531,1616648566,1616653669',
    '__jsl_clearance_s': '1617787467.471|0|TOtWf5Uh4ztWeoWW9mZKHM70cK0%3D',
    'security_session_verify': '0739a344b878c9a797e3fc5e1f67547c',
    'insert_cookie': '65472086',
}


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:87.0) Gecko/20100101 Firefox/87.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Referer': 'https://eng.yidaiyilu.gov.cn/',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Cache-Control': 'max-age=0',
}

url = 'https://eng.yidaiyilu.gov.cn/'
home_page = requests.get(url, headers=headers, timeout=120)
home_page = requests.get(url, headers=headers, timeout=120, cookies=cookies).text

home_pages=c.scrap('id="n3"(.*?)id="n9"',home_page)
for _ in home_pages.split('<li>')[1:]:
    cat_url = c.scrap('href="(.*?)"', _)
    
    if '#' in cat_url:
        continue
    if '10077' in cat_url:
        continue
    if '10075' in cat_url:
        continue
    if '10161' in cat_url:
        continue
    if 'htm' in cat_url: #1072,76
        continue
    if 'gbs_id=2' in cat_url:
        continue
    if '10072' in cat_url:
        continue
    if '10076' in cat_url:
        continue     
    if 'http' not in cat_url:
        cat_url='https://eng.yidaiyilu.gov.cn'+cat_url
    logger.info(f'Fetching url {cat_url}\n')
    cat_page = requests.get(cat_url, headers=headers,cookies=cookies).text
    
    if cat_page.startswith('Unable to fetch'):
        logger.info(cat_page) # writes error message with error code
        unable_to_fetch_rss_url += 1
        continue
        
    cat_pages=c.scrap('<ul\s*class="commonList_dot.*?">(.*?)</ul>',cat_page)
    
    for i in cat_pages.split('href=')[1:]:
        source_link = c.scrap('"(.*?)"', i)
        if 'http' not in source_link:
            source_link='https://eng.yidaiyilu.gov.cn'+source_link
        page=requests.get(source_link, headers=headers,cookies=cookies).text
        
        if page.startswith('Unable to fetch'):
                logger.info(page) # writes error message with error code
                unable_to_fetch_article_url += 1
                continue   
        # handle duplicates
        source_link_query = {'source_link':source_link}
        dic = cl_data.find_one(source_link_query,{'source_link': 1}) 
        if dic:
            duplicate_data += 1
            continue

        time.sleep(random.randint(1,3))
       
        # source_headline
        source_headline = c.scrap('<title>(.*?)</title>', page)
        
        # skip if headline not found
        if not source_headline:
            logger.info(f'Skipping due to headline {source_link}\n')
            skipped_due_to_headline += 1
            continue
         # Date and time
        pub_date, publish_time = '', ''

        try:
            date_time_str = c.scrap('<meta\s*name="others"\s*content="(.*?)"', page) 
            date_time_str = re.sub('[^\w+]', '', date_time_str)
            date_time_obj = datetime.strptime(date_time_str, '%Y%m%d%H%M%S')
            # China is 2 hours and 30 minutes ahead of India
            ist_date_time = date_time_obj - timedelta(hours = 2,minutes = 30)  
            ist_date_time = ist_date_time.strftime('%Y-%m-%d %H:%M:%S')
            pub_date = ist_date_time[:10]
            publish_time = ist_date_time[11:]
        except:
            pass
        # skip null date
        if not pub_date:
            logger.info(f'Skipping due to date {source_link}\n')
            skipped_due_to_date += 1
            continue

        # break if date is not today's date
        if pub_date != created_on:
            break     

        # source_content          
        source_content = c.scrap('<div\s*class="info_content">(.*?)<div\s*class="page"', page)

        source_content = c.strip_html(source_content)
        if not  source_content:
            source_content=c.scrap('<div\s*id="galleryTp.*?>(.*?)class="editor">',page)
            source_content=source_content.replace('<div','',re.S)
            source_content = c.strip_html(source_content)
        source_content = c.strip_html(source_content)
       
        # skip if content not found
        if not source_content:
            logger.info(f'Skipping due to content {source_link}\n')
            # print(f'Skipping due to content {source_link}\n')
            skipped_due_to_content += 1
            continue

        # journalist
        journalist =  ''
        if not journalist: journalist = 'NA'

        # current date and time 
        harvest_time = datetime.now().strftime("%H:%M:%S")

        # temp link
        temp_link = source_link

        # headline and content 
        headline = source_headline
        content = source_content

        # overall_tonality
        overall_tonality = ''

        # word count
        word_count = len((source_headline + ' ' + source_content).split())

        html_content = ''

        # image_urls
        image_urls = []

        # storing the above data in a dictionary
        clientdata ={
                        "client_master" : client_id, 
                        "articleid":client_id,
                        "medium":'Web' ,
                        "searchkeyword":[],
                        "entityname" : [] ,
                        "process_flage":"1",
                        "na_flage":"0",
                        "na_reason":"",
                        "qc_by":"",
                        "qc_on":"",
                        "location":"",
                        "spokeperson":"",
                        "quota":"",
                        "overall_topics":"",
                        "person":"",
                        "overall_entites":"",
                        "overall_tonality": overall_tonality,
                        "overall_wordcount":word_count,
                        "article_subjectivity":"",
                        "article_summary":"",
                        "pub_date":pub_date,
                        "publish_time":publish_time,
                        "harvest_time":harvest_time,
                        "temp_link":temp_link,
                        "publish_source": publish_source,
                        "programme":'null',
                        "feed_class":"News",
                        "publishing_platform":"",
                        "klout_score":"",
                        "journalist":journalist,
                        "headline":headline,
                        "content":content,
                        "source_headline":source_headline,
                        "source_content":source_content,
                        "language":language,
                        "presence":'null',
                        "clip_type":'null',
                        "prog_slot":'null',
                        "op_ed":'0',
                        "location_mention":'',
                        "source_link":source_link,
                        "author_contact":'',
                        "author_emailid":'',
                        "author_url":'',
                        "city":'',
                        "state":'',
                        "country":country,
                        "source":publish_source,
                        "foot_fall":'',
                        "created_on":created_on,
                        "active":'1',
                        'crawl_flag':2,
                        "images_path":images_path,
                        "html_content":html_content
                    } 

#         cl_data.insert_one(clientdata)  
        no_of_data += 1


logger.info('Iteration complete\n')   

logger.info(f'Number of data: {no_of_data}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to fetch rss url: {unable_to_fetch_rss_url}\n')
logger.info(f'Unable to fetch article url: {unable_to_fetch_article_url}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n')